# ONETEP Geometry Optimization with ASE

This notebook demonstrates how to run ONETEP geometry optimizations using the ASE interface, following the steps in the ONETEP tutorial.

You may need to install the newest version of ASE to run this tutorial, if everything is already setup, to upgrade to the lastest version, you should run

```bash
pip install --force-reinstall --no-deps git+https://gitlab.com/ase/ase.git@master
```

The "--no-deps" flag is used to avoid reinstalling the dependencies if you already have them.

Please have a look at tutorial 2 before running the jupyter examples, they contain information about the ONETEP input files and the pseudopotentials used.

You might need to change the path to the ONETEP command and pseudopotential path in the `OnetepProfile`. Please make sure that the script launched contain the proper module and environment setup, as adviced in the `_static/launch_onetep.sh` file.

In [ ]:
from pathlib import Path

from ase.build import molecule
from ase.calculators.onetep import Onetep, OnetepProfile
from ase.io import read
from ase.io.onetep import get_onetep_keywords
from ase.visualize import view

In [ ]:
profile = OnetepProfile(
    command="../_static/launch_onetep.sh", pseudo_path="../_static/pseudos"
)

## Ethene Geometry Optimization

Set up the ethene molecule and ONETEP calculator:

In [ ]:
ethene = molecule("C2H4", vacuum=8.0)

calc_dir = Path("t4_onetep_ethene")
calc_dir.mkdir(exist_ok=True)

keywords = {
    "xc_functional": "PBE",
    "output_detail": "verbose",
    "cutoff_energy": "650 eV",
    "task": "GeometryOptimization",
    "output_detail": "verbose",
}

pseudos = {"H": "hydrogen.recpot", "C": "carbon.recpot"}

calc = Onetep(
    keywords=keywords,
    pseudopotentials=pseudos,
    ngwf_radius=6.0,
    ngwf_count={"H": 1, "C": 4},
    profile=profile,
    directory=calc_dir,
)

ethene.calc = calc

ethene.get_potential_energy()

## Sucrose Geometry Optimization 

Set up the sucrose molecule and ONETEP calculator:

Here we use a specific conformer of the sucrose, which ID was retrieved from the [PubChem](https://pubchem.ncbi.nlm.nih.gov/) database.

In [ ]:
from ase.data.pubchem import pubchem_atoms_search

sucrose = pubchem_atoms_search(conformer="0000176400000007")
sucrose.center(vacuum=5.0)

calc_dir = Path("t4_onetep_sucrose")
calc_dir.mkdir(exist_ok=True)

keywords = {
    "task": "GEOMETRYOPTIMIZATION",
    "output_detail": "verbose",
    "cutoff_energy": "700.0 eV",
    "read_denskern": False,
    "write_denskern": False,
    "read_tightbox_ngwfs": False,
    "write_tightbox_ngwfs": True,
    "write_converged_dkngwfs": True,
    "geom_disp_tol": 1e-2,
}

pseudos = {
    "H": "hydrogen.recpot",
    "C": "carbon.recpot",
    "O": "oxygen.recpot",
}

calc = Onetep(
    label="sucrose",
    keywords=keywords,
    pseudopotentials=pseudos,
    ngwf_count={"H": 1, "C": 4, "O": 4},
    ngwf_radius=6.75,
    directory=calc_dir,
    profile=profile,
)

sucrose.calc = calc

# Run the geometry optimization
sucrose.get_potential_energy()

sucrose_opt = read(f"{calc_dir}/onetep.out")
v = view([sucrose, sucrose_opt], viewer="ngl")
v.view.add_ball_and_stick()
v

This sets up geometry optimizations for ethene and sucrose molecules using ONETEP via the ASE interface. Key steps include:

1. Building the molecules with ASE.
2. Setting up the ONETEP calculator with keywords and pseudopotentials.
3. Attaching the calculator to the Atoms object .
4. Running the optimization with ONETEP internal BFGS.

Refer to the full tutorial for more details on monitoring and analyzing the optimization.